In [ ]:
import psycopg2
import csv

# Подключение к базе данных PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="phonebookdb",  # имя базы данных
    user="postgres",
    password="12345678",
    port=5432
)

cur = conn.cursor()

# Удаляем и пересоздаем таблицу phonebookdb
cur.execute("DROP TABLE IF EXISTS phonebookdb")
cur.execute("""
    CREATE TABLE phonebookdb (
        name TEXT,
        Phone_number TEXT
    )
""")
conn.commit()

# SQL-запросы
sqlIN = 'INSERT INTO phonebookdb(name, Phone_number) VALUES (%s, %s) RETURNING *'
sqlDEL = 'DELETE FROM phonebookdb WHERE name = %s'
sqlUPDATEnum = 'UPDATE phonebookdb SET Phone_number = %s WHERE name = %s'
sqlUPDATEname = 'UPDATE phonebookdb SET name = %s WHERE Phone_number = %s'
sqlCHECK = 'SELECT * FROM phonebookdb WHERE name = %s'
sqlCHECKN = 'SELECT * FROM phonebookdb WHERE Phone_number = %s'

# Чтение из CSV
def readcsv(filename):
    data = []
    with open(filename, 'r') as f:
        datar = csv.reader(f, quotechar='"')
        for row in datar:
            if len(row) == 2:
                data.append((row[0], row[1]))
    print("CSV Data:", data)
    return data

# Меню
print('to insert manually type 1 \nto insert from csv type 2')
print('to delete record type 3')
print('to find the number type 4')

choice = int(input("Your choice: "))

if choice == 1:
    name = input("Enter name: ")
    number = input("Enter number: ")
    if len(number) == 11:
        cur.execute(sqlCHECK, (name,))
        result = cur.fetchone()

        cur.execute(sqlCHECKN, (number,))
        Nresult = cur.fetchone()

        if result:
            cur.execute(sqlUPDATEnum, (number, name))
            print("Phone number updated.")
        elif Nresult:
            cur.execute(sqlUPDATEname, (name, number))
            print("Name updated.")
        else:
            cur.execute(sqlIN, (name, number))
            print("Inserted new record.")
        conn.commit()
    else:
        print('Invalid number (must be 11 digits)')

elif choice == 2:
    data = readcsv('LegendsPhoneBook.csv')
    cur.executemany(sqlIN, data)
    conn.commit()
    print("CSV data inserted.")

elif choice == 3:
    name = input('Type name: ')
    cur.execute(sqlCHECK, (name,))
    result = cur.fetchone()
    if result:
        cur.execute(sqlDEL, (name,))
        conn.commit()
        print('Deleted')
    else:
        print('No such record')

elif choice == 4:
    name = input('Enter name to search: ')
    cur.execute(sqlCHECK, (name,))
    result = cur.fetchone()
    if result:
        print('Found:', result)
    else:
        print('No record found')

else:
    print("Invalid choice.")

cur.close()
conn.close()


to insert manually type 1 
to insert from csv type 2
to delete record type 3
to find the number type 4
